# SAMN'S MERCADO: END TO END E-COMMERCE PLATFORM
---
LEARNING TEAM 3 - ASIS, ITUCAL, MERCADO, RIZADA

## Introduction
This project will tackle an e-commerce platform that will:
- Provide diversified selection of goods to customers and transact online
- Allow vendors to provide selection of goods to customers and provide analytics platform to observe sales performance

The end to end solution will include:
- API interfaces for data upload 
- OLTP processing (DynamoDB+Postgres)
- Orchestration and ETL/ELT
- Data Lakehouse
- Storage Layers
- RBAC (Role Based Access Control)
- OLTP Data Warehouse
- Consumption

Limitations:
- API endpoints has no authentication
- API will be mostly POST endpoints

## Architecture Overview

In [ ]:
%load_ext sql

In [ ]:
import boto3
import requests
import os
import pandas as pd
from sqlalchemy import create_engine
from faker import Faker
import redshift_connector

In [ ]:
pd.set_option('display.max_rows', 12)
pd.set_option('display.min_rows', 6)

In [ ]:
fake = Faker()

base_url = "http://fastapi:8000"
headers = {"Content-Type": "application/json"}

print("customers")
# Create customers
for customer_id in range(1, 11):
    data = {
        "first_name": fake.first_name(),
        "last_name": fake.last_name(),
        "email": fake.email(),
        "joined_at": fake.iso8601()
    }
    response = requests.post(f"{base_url}/customers/", json=data, headers=headers)
    print(response.status_code, response.json())

print("vendors")
# Create vendors
for vendor_id in range(1, 6):
    data = {
        "vendor_name": fake.company(),
        "region": fake.country_code()
    }
    response = requests.post(f"{base_url}/vendors/", json=data, headers=headers)
    print(response.status_code, response.json())

print("inventory per vendor")
# Add inventory per vendor
for vendor_id in range(1, 6):
    for item_id in range(1, 11):
        params = {"vendor_id": vendor_id}
        data = {
            "item_name": fake.word(),
            "category": str(1),
            "price": fake.random_int(min=10, max=500),
        }
        response = requests.post(f"{base_url}/inventory/", json=data, headers=headers, params=params)
        print(response.status_code, response.json())

print("cart and checkout twice and add to cart again!")
for customer_id in range(1, 11):
    for _ in range(2):  # Checkout twice
        price1 = fake.random_int(min=10, max=500)
        price2 = fake.random_int(min=10, max=500)
        qty1 = fake.random_int(min=10, max=500)
        qty2 = fake.random_int(min=10, max=500)
        cart_data = {
            "user_id": customer_id,
            "cart": [
                {
                    "item_id": str(fake.random_int(min=1, max=10)),
                    "vendor_id": str(fake.random_int(min=1, max=5)),
                    "qty": qty1,
                    "unit_price": price1,
                    "total_price": qty1*price1,
                },
                {
                    "item_id": str(fake.random_int(min=1, max=10)),
                    "vendor_id": str(fake.random_int(min=1, max=5)),
                    "qty": qty2,
                    "unit_price": price2,
                    "total_price": qty2*price2,
                }
            ]
        }
        response = requests.post(f"{base_url}/cart/", json=cart_data, headers=headers)
        print(response.status_code, response.json())
        
        response = requests.post(f"{base_url}/checkout/", json={"user_id": customer_id}, headers=headers)
        print(response.status_code, response.text)

    price1 = fake.random_int(min=10, max=500)
    price2 = fake.random_int(min=10, max=500)
    qty1 = fake.random_int(min=10, max=500)
    qty2 = fake.random_int(min=10, max=500)
    # Add to cart one more time
    cart_data = {
        "user_id": customer_id,
        "cart": [
            {
                "item_id": str(fake.random_int(min=1, max=10)),
                "vendor_id": str(fake.random_int(min=1, max=5)),
                "qty": qty1,
                "unit_price": price1,
                "total_price": qty1*price1,
            },
            {
                "item_id": str(fake.random_int(min=1, max=10)),
                "vendor_id": str(fake.random_int(min=1, max=5)),
                "qty": qty1,
                "unit_price": price1,
                "total_price": qty1*price1,
            }
        ]
    }
    response = requests.post(f"{base_url}/cart/", json=cart_data, headers=headers)
    print(response.status_code, response.json())

In [ ]:
OLTP_USER = os.getenv("POSTGRES_USER")
OLTP_PASS = os.getenv("POSTGRES_PASSWORD")
OLTP_HOST = os.getenv("POSTGRES_HOST")
OLTP_DB_NAME = os.getenv("POSTGRES_DB")
OLAP_USER = os.getenv("REDSHIFT_USER")
OLAP_PASS = os.getenv("REDSHIFT_PASSWORD")
OLAP_HOST = os.getenv("REDSHIFT_HOST")
OLAP_DB_NAME = os.getenv("REDSHIFT_DB")

In [ ]:
connection_string = f"postgresql://{OLTP_USER}:{OLTP_PASS}@{OLTP_HOST}:5432/{OLTP_DB_NAME}"
engine = create_engine(connection_string)
get_ipython().run_line_magic('sql', connection_string)

In [ ]:
%sql \dt

In [ ]:
table_list = [
    "customer", "inventory", "vendor"
]

In [ ]:
for table in table_list:
    print(f"{table}")
    schema_query = f"columns --table {table}"
    display(get_ipython().run_line_magic('sqlcmd', schema_query))

In [ ]:
for table in table_list:
    print(f"{table}")
    display(pd.read_sql(f"select * from {table} limit 100", engine.raw_connection()))

In [ ]:
connection_string = f"postgresql://{OLAP_USER}:{OLAP_PASS}@{OLAP_HOST}:5439/{OLAP_DB_NAME}"
engine = redshift_connector.connect(
    host=OLAP_HOST,
    port=5439,
    database=OLAP_DB_NAME,
    user=OLAP_USER,
    password=OLAP_PASS
)
engine = create_engine(connection_string)
get_ipython().run_line_magic('sql', connection_string)

In [ ]:
%sql \dt

In [ ]:
table_list = [
    "category", "customer", "date", "inventory", "vendor", "sales"
]

In [ ]:
for table in table_list:
    print(f"{table}")
    schema_query = f"columns --table {table}"
    display(get_ipython().run_line_magic('sqlcmd', schema_query))

In [ ]:
for table in table_list:
    print(f"{table}")
    display(pd.read_sql(f"select * from {table} limit 100", engine.raw_connection()))